In [1]:
import csv
import mysql.connector
from mysql.connector import Error
from datetime import datetime

In [2]:
# Conexión a MySQL
config = {
    'user': 'root', # nombre de usuario
    'password': 'admin', # contraseña
    'host': 'localhost',
    'database': 'coches', # nombre de la base de datos
    'raise_on_warnings': True
}

def create_connection():
    """Crear conexión a la base de datos"""
    try:
        conn = mysql.connector.connect(**config)
        return conn
    except Error as e:
        print(f"Error al conectar a MySQL: {e}")
        return None

def insert_ubicacion(conn, ubicacion, latitud=None, longitud=None):
    """Insertar ubicación si no existe y devolver ID"""
    cursor = conn.cursor()
    try:
        # Verificar si ya existe
        cursor.execute("SELECT id FROM ubicaciones WHERE ubicacion = %s", (ubicacion,))
        result = cursor.fetchone()
        
        if result:
            return result[0]
        else:
            # Insertar nueva ubicación
            cursor.execute("""
                INSERT INTO ubicaciones (ubicacion, latitud, longitud) 
                VALUES (%s, %s, %s)
            """, (ubicacion, latitud, longitud))
            conn.commit()
            return cursor.lastrowid
    except Error as e:
        print(f"Error al insertar ubicación: {e}")
        return None
    finally:
        cursor.close()

def insert_tipo_carroceria(conn, tipo_carroceria):
    """Insertar tipo de carrocería si no existe y devolver ID"""
    if not tipo_carroceria or tipo_carroceria == 'Desconocido':
        return None
        
    cursor = conn.cursor()
    try:
        # Verificar si ya existe
        cursor.execute("SELECT id FROM tipos_carroceria WHERE tipo_carroceria = %s", (tipo_carroceria,))
        result = cursor.fetchone()
        
        if result:
            return result[0]
        else:
            # Insertar nuevo tipo
            cursor.execute("INSERT INTO tipos_carroceria (tipo_carroceria) VALUES (%s)", (tipo_carroceria,))
            conn.commit()
            return cursor.lastrowid
    except Error as e:
        print(f"Error al insertar tipo de carrocería: {e}")
        return None
    finally:
        cursor.close()

def insert_combustible(conn, combustible):
    """Insertar tipo de combustible si no existe y devolver ID"""
    if not combustible or combustible == 'Desconocido':
        return None
        
    cursor = conn.cursor()
    try:
        # Verificar si ya existe
        cursor.execute("SELECT id FROM combustibles WHERE combustible = %s", (combustible,))
        result = cursor.fetchone()
        
        if result:
            return result[0]
        else:
            # Insertar nuevo tipo
            cursor.execute("INSERT INTO combustibles (combustible) VALUES (%s)", (combustible,))
            conn.commit()
            return cursor.lastrowid
    except Error as e:
        print(f"Error al insertar tipo de combustible: {e}")
        return None
    finally:
        cursor.close()

def insert_transmision(conn, transmision):
    """Insertar tipo de transmisión si no existe y devolver ID"""
    if not transmision or transmision == 'Desconocido':
        return None
        
    cursor = conn.cursor()
    try:
        # Verificar si ya existe
        cursor.execute("SELECT id FROM transmisiones WHERE transmision = %s", (transmision,))
        result = cursor.fetchone()
        
        if result:
            return result[0]
        else:
            # Insertar nuevo tipo
            cursor.execute("INSERT INTO transmisiones (transmision) VALUES (%s)", (transmision,))
            conn.commit()
            return cursor.lastrowid
    except Error as e:
        print(f"Error al insertar tipo de transmisión: {e}")
        return None
    finally:
        cursor.close()

def to_boolean(value):
    """Convertir valor a booleano"""
    if isinstance(value, bool):
        return value
    if isinstance(value, (int, float)):
        return bool(value)
    if isinstance(value, str):
        return value.lower() in ('true', '1', 't', 'y', 'yes', 'sí', 'si')
    return False

def to_float(value):
    """Convertir valor a float seguro"""
    try:
        return float(value) if value and str(value).strip() else None
    except (ValueError, TypeError):
        return None

def to_int(value):
    """Convertir valor a int seguro"""
    try:
        return int(float(value)) if value and str(value).strip() else None
    except (ValueError, TypeError):
        return None

def process_csv_file(csv_file):
    """Procesar el archivo CSV y cargar los datos en la base de datos"""
    conn = create_connection()
    if not conn:
        return
    
    cursor = conn.cursor(dictionary=True)
    
    with open(csv_file, mode='r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        
        for row in csv_reader:
            try:
                # Procesar ubicación
                ubicacion_id = insert_ubicacion(
                    conn, 
                    row['ubicacion'],
                    to_float(row.get('latitud')),
                    to_float(row.get('longitud'))
                )
                
                # Procesar tipo de carrocería
                carroceria_id = insert_tipo_carroceria(conn, row.get('tipo_carroceria'))
                
                # Procesar combustible
                combustible_id = insert_combustible(conn, row.get('combustible'))
                
                # Procesar transmisión
                transmision_id = insert_transmision(conn, row.get('transmision'))
                
                # Preparar datos del vehículo
                vehiculo_data = {
                    'id_extraccion': row.get('id_extraccion'),
                    'timestamp_extraccion': row.get('timestamp_extraccion'),
                    'marca': row.get('marca'),
                    'modelo': row.get('modelo'),
                    'precio_contado': to_float(row.get('precio_contado')),
                    'año_matriculacion': to_int(row.get('año_matriculacion')),
                    'combustible_id': combustible_id,
                    'kilometraje': to_int(row.get('kilometraje')),
                    'ubicacion_id': ubicacion_id,
                    'latitud': to_float(row.get('latitud')),
                    'longitud': to_float(row.get('longitud')),
                    'garantia': row.get('garantia'),
                    'transmision_id': transmision_id,
                    'potencia_cv': to_int(row.get('potencia_cv')),
                    'puertas': to_int(row.get('puertas')),
                    'asientos': to_int(row.get('asientos')),
                    'tipo_carroceria_id': carroceria_id,
                    'mes_matriculacion': to_int(row.get('mes_matriculacion')),
                    'financiacion_disponible': to_boolean(row.get('financiacion_disponible')),
                    'es_km0': to_boolean(row.get('es_km0')),
                    'es_demo': to_boolean(row.get('es_demo'))
                }
                
                # Insertar vehículo
                insert_vehiculo = """
                INSERT INTO vehiculos (
                    id_extraccion, timestamp_extraccion, marca, modelo, precio_contado,
                    año_matriculacion, combustible_id, kilometraje, ubicacion_id,
                    latitud, longitud, garantia, transmision_id, potencia_cv,
                    puertas, asientos, tipo_carroceria_id, mes_matriculacion,
                    financiacion_disponible, es_km0, es_demo
                ) VALUES (
                    %(id_extraccion)s, %(timestamp_extraccion)s, %(marca)s, %(modelo)s, %(precio_contado)s,
                    %(año_matriculacion)s, %(combustible_id)s, %(kilometraje)s, %(ubicacion_id)s,
                    %(latitud)s, %(longitud)s, %(garantia)s, %(transmision_id)s, %(potencia_cv)s,
                    %(puertas)s, %(asientos)s, %(tipo_carroceria_id)s, %(mes_matriculacion)s,
                    %(financiacion_disponible)s, %(es_km0)s, %(es_demo)s
                )
                """
                
                cursor.execute(insert_vehiculo, vehiculo_data)
                conn.commit()
                
            except Exception as e:
                print(f"Error al procesar fila: {e}")
                print(f"Fila problemática: {row}")
                conn.rollback()
    
    cursor.close()
    conn.close()
    print("Proceso de carga de datos completado.")

# Ejecutar el proceso
if __name__ == "__main__":
    csv_file = "C:\\Users\\abner\\proyecto\\Proyecto-\\DB\\datos_limpios.csv"
    process_csv_file(csv_file)

Proceso de carga de datos completado.
